# Lab 9

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Thomatous/geog510-labs/blob/main/lab_09.ipynb)

In [ ]:
%pip install -q -U leafmap
%pip install -q fiona

In [ ]:
import leafmap
import leafmap.foliumap as foliumap
import geopandas as gpd



## Question 1: Creating an Interactive Map

1. Create an interactive map with search functionality that allows users to search for places and zoom to them. Disable the draw control on the map.

In [ ]:
m = leafmap.Map(
    center=(40, -100),
    zoom=4,
    height="600px",
    draw_control=False,
)
url = "https://nominatim.openstreetmap.org/search?format=json&q={s}"
m.add_search_control(url, zoom=10, position="topleft")
m

## Question 2: Adding Map Legends

1. Add the [ESA World Cover](https://esa-worldcover.org/en) WMS tile layer to the map.
   - URL: `https://services.terrascope.be/wms/v2?`
   - Layer name: `WORLDCOVER_2021_MAP`
2. Add a legend to the map using the leafmap built-in `ESA_WorldCover` legend.

In [ ]:
m = foliumap.Map(center=[40, -100], zoom=4) # NOTE: the default leafmap backend doesn't seem to display the legend correctly

url = "https://services.terrascope.be/wms/v2?"
m.add_wms_layer(
    url=url,
    layers="WORLDCOVER_2021_MAP",
    name="WorldCover 2021",
    attribution="ESA",
    format="image/png",
    shown=True,
)

m.add_legend(
    title="ESA WorldCover (2021)",
    builtin_legend="ESA_WorldCover",
    position="bottomright",
)
m

## Question 3: Creating Marker Clusters

1. Create a marker cluster visualization from a GeoJSON file of building centroids:
   - URL: https://github.com/opengeos/datasets/releases/download/places/wa_building_centroids.geojson
   - Hint: Read the GeoJSON file using GeoPandas and add "latitude" and "longitude" columns to the GeoDataFrame.
2. Create circle markers for each building centroid using the `Map.add_circle_markers_from_xy()` method with the following styling:
   - Radius: 5
   - Outline color: "red"
   - Fill color: "yellow"
   - Fill opacity: 0.8

In [ ]:
def read_geojson_and_add_longtitude_and_attitude(url: str) -> gpd.GeoDataFrame:
    gdf = gpd.read_file(url)
    gdf["longitude"] = gdf.geometry.x
    gdf["latitude"] = gdf.geometry.y
    return gdf

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/places/wa_building_centroids.geojson"
gdf = read_geojson_and_add_longtitude_and_attitude(url)

m1 = leafmap.Map(center=[47.5, -120], zoom=7)
m1.add_marker_cluster(gdf, x="longitude", y="latitude", layer_name="Buildings")
m1

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/places/wa_building_centroids.geojson"
gdf = read_geojson_and_add_longtitude_and_attitude(url)

m2 = leafmap.Map(center=[47.5, -120], zoom=7)
m2.add_circle_markers_from_xy(
    gdf,
    x="longitude",
    y="latitude",
    radius=5,
    color="red",
    fill_color="yellow",
    fill_opacity=0.8,
    layer_name="Buildings"
)
m2

## Question 4: Visualizing Vector Data

1. Visualize the building polygons GeoJSON file and style it with:

   - Outline color: "red"
   - No fill color
   - URL: https://github.com/opengeos/datasets/releases/download/places/wa_overture_buildings.geojson

2. Visualize the road polylines GeoJSON file and style it with:

   - Line color: "red"
   - Line width: 2
   - URL: https://github.com/opengeos/datasets/releases/download/places/las_vegas_roads.geojson

3. Create a choropleth map of county areas in the US:
   - URL: https://github.com/opengeos/datasets/releases/download/us/us_counties.geojson
   - Column: `CENSUSAREA`

In [ ]:
def read_geojson_and_explore(url: str) -> gpd.GeoDataFrame:
    gdf = gpd.read_file(url)
    gdf.explore()
    return gdf

In [ ]:
url="https://github.com/opengeos/datasets/releases/download/places/wa_overture_buildings.geojson"
gdf = read_geojson_and_explore(url)

m = leafmap.Map(center=[47.5, -120], zoom=7)
m.add_basemap("Esri.WorldImagery")
style = {"color": "red", "fillOpacity": 0}
m.add_gdf(gdf, style=style, layer_name="Building Outlines", zoom_to_layer=True)
m

In [ ]:
url="https://github.com/opengeos/datasets/releases/download/places/las_vegas_roads.geojson"
gdf = read_geojson_and_explore(url)

m = leafmap.Map(center=[47.5, -120], zoom=7)
m.add_basemap("Esri.WorldImagery")
style = {"color": "red",  "weight": 2}
m.add_gdf(gdf, style=style, layer_name="Road Polygons", zoom_to_layer=True)
m

In [ ]:

data = "https://github.com/opengeos/datasets/releases/download/us/us_counties.geojson"
m = leafmap.Map()
m.add_data(
    data,
    column="CENSUSAREA",
    cmap="Blues",
    scheme="Quantiles",
)
m

## Exercise 5: Creating a Split Map

1. Create a split map to compare imagery of Libya before and after the 2023 flood event:

   - Pre-event imagery: https://github.com/opengeos/datasets/releases/download/raster/Libya-2023-07-01.tif
   - Post-event imagery: https://github.com/opengeos/datasets/releases/download/raster/Libya-2023-09-13.tif

In [ ]:
m = leafmap.Map(center=[39.494897, -108.507278], zoom=10)
url = "https://github.com/opengeos/datasets/releases/download/raster/Libya-2023-07-01.tif"
url2 = "https://github.com/opengeos/datasets/releases/download/raster/Libya-2023-09-13.tif"
m.split_map(
    left_layer=url, right_layer=url2, left_label="Pre-event", right_label="Post-event"
)
m